# Gai/Gen: Image-to-Text (ITT)

## 1. Note

The following examples has been tested on the following environment:
-   Ubuntu 22.04
-   Python 3.10
-   CUDA Toolkit 11.8
-   openai 1.6.1
-   llava 1.1.3

## 2. Create Virtual Environment and Install Dependencies

We will create a seperate virtual environment for this to avoid conflicting dependencies that each underlying model requires.

```sh
sudo apt update -y && sudo apt install ffmpeg git git-lfs -y
conda create -n ITT python=3.10.10 -y
conda activate ITT
pip install gai-gen[ITT]
```

## 3. Examples

In [2]:
## 6.17 OpenAI Vision Image-to-Text Generation

from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('openai-vision')


import base64
encoded_string = ""
with open("../tests/buses.jpeg", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

print("GENERATE:")
response = gen.create(
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image?"},
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encoded_string}",
          },
        },
      ],
    }
  ],
  max_tokens=300
  )
print(response.choices[0].message.content)


GENERATE:
The image shows a bus stop sign, with various elements providing information to passengers. There's a blue rectangular sign at the top with a bus icon, symbolizing that it's a bus stop. Below it, there is a sign with a bus stop number (71091) and what appears to be a code (BIK 637).

Beside these signs, there is a schedule board with lists of numbers in red and white, which are likely the bus service numbers that stop at this location. The numbers are organized in a grid, possibly grouped by the route or schedule, to inform passengers about their options for public transportation at this stop.

In the background, there are trees, indicating that the bus stop is situated in an area with greenery. The overall scene suggests this is a bus stop intended for regular public use, possibly in an urban or suburban area.


In [3]:
## 6.18 OpenAI Vision Image-to-Text Streaming

from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('openai-vision')

import base64
encoded_string = ""
with open("../tests/buses.jpeg", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

print("STREAMING:")
response = gen.create(
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image?"},
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encoded_string}",
          },
        },
      ],
    }
  ],
  max_tokens=300,
  stream=True
  )
for chunk in response:
    if chunk.choices[0].delta.content:
      print(chunk.choices[0].delta.content,end="",flush=True)


STREAMING:
The image displays a bus stop sign, indicating the location where buses pick up passengers. The sign shows several elements:

1. A blue rectangle with a white bus symbol on top, clearly marking that the sign is for a bus stop.
2. A sign with a bus stop number "71091", along with "Blk 637" which likely refers to the nearby building or block number.
3. A list of bus numbers on panels underneath, showing the different bus services that stop at this location. 
4. Two direction arrows, possibly indicating the direction in which the buses will be going, or the locations of nearby bus stops.

The greenery in the background suggests that the bus stop is located in an area with trees, likely providing some shade for waiting passengers.

The following demo is uses the LLaVa model, installation instructions can be found [here](./docs/8-llava-transformers.md).

In [5]:
## 6.19 LlaVa Image-to-Text Generation

from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('llava-transformers')

import base64
encoded_string = ""
with open("../tests/buses.jpeg", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

print("GENERATE:")
response = gen.create(
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image?"},
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encoded_string}",
          },
        },
      ],
    }
  ],
  max_new_tokens=300
  )
print(response.choices[0].message.content)


ModuleNotFoundError: No module named 'llava'

In [1]:
## 6.20 LlaVa Image-to-Text Streaming

from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('llava-transformers')

import base64
encoded_string = ""
with open("../tests/buses.jpeg", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

print("STREAMING:")
response = gen.create(
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image?"},
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encoded_string}",
          },
        },
      ],
    }
  ],
  max_new_tokens=300,
  stream=True
  )
for chunk in response:
    print(chunk.choices[0].delta.content,end="",flush=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

STREAMING:
The image features a street scene with a bus stop sign and a bus route sign. The bus stop sign is located on a pole, while the bus route sign is hanging on the side of the pole. The bus route sign displays the number 7101, indicating the bus route.

There are several trees in the background, adding to the urban atmosphere. A person can be seen in the scene, possibly waiting for the bus or just passing by.by.

## Running as a Service

### Step 1: Start Docker container

```bash
docker run -d \
    --name gai-itt \
    -p 12031:12031 \
    --gpus all \
    -v ~/gai/models:/app/models \
    kakkoii1337/gai-itt:latest
```

### Step 2: Wait for model to load

```bash
docker logs gai-itt
```

When the loading is completed, the logs should show this:

```bash
INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:12031 (Press CTRL+C to quit)
```

### Step 3: Test

In [1]:
%%bash
curl -X POST http://localhost:12031/gen/v1/vision/completions \
    -H "Content-Type: application/json" \
    -N -s \
    -d "{\"max_new_tokens\":1000,\"stream\":true,\"messages\":[{\"role\":\"user\",\"content\":[{\"type\": \"text\", \"text\": \"What’s in this image?\"},{\"type\":\"image_url\",\"image_url\":{\"url\":\"...\"}}]}]}"

{"id": "7125d181-16bc-452a-ad30-4e52c2d13ee2", "choices": [{"delta": {"content": "", "function_call": null, "role": "assistant", "tool_calls": null}, "finish_reason": null, "index": 0, "logprobs": null, "message": ""}], "created": 1703793141, "model": "Llava v1.5", "object": "chat.completion.chunk", "system_fingerprint": null, "usage": null}
{"id": "7125d181-16bc-452a-ad30-4e52c2d13ee2", "choices": [{"delta": {"content": "The ", "function_call": null, "role": "assistant", "tool_calls": null}, "finish_reason": null, "index": 0, "logprobs": null, "message": "The "}], "created": 1703793143, "model": "Llava v1.5", "object": "chat.completion.chunk", "system_fingerprint": null, "usage": null}
{"id": "7125d181-16bc-452a-ad30-4e52c2d13ee2", "choices": [{"delta": {"content": "image ", "function_call": null, "role": "assistant", "tool_calls": null}, "finish_reason": null, "index": 0, "logprobs": null, "message": "image "}], "created": 1703793144, "model": "Llava v1.5", "object": "chat.completion